In [14]:
import requests
import random
import json
import tqdm

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'

DATA    = {
    "message"   : "How do you treat peach leaf curl?",
    "sender"    : str(random.randint(0, 100000))
}

r = requests.post(URL, json = DATA)

if r.status_code == 200:
    print(json.loads(r.text))
else:
    print('Service unavailable')

[{'recipient_id': '26698', 'text': 'Extracted slots</br>[Format: (<i>relation</i>) <strong>entity</strong> - <strong>value</strong>]:</br>Group 1:</br>(<i>pest</i>) <strong>name</strong> - <strong>peach leaf curl</strong></br>(<i>pest</i>) <strong>action</strong> - <strong>treat</strong></br></br>Composed 1 additional queries:</br>1) <i>peach leaf curl treat</i></br>'}, {'recipient_id': '26698', 'text': 'Results with slots improvement... Top 10 results.'}, {'recipient_id': '26698', 'custom': {'text': 'Here are my top results:', 'payload': 'collapsible', 'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(score: 0.89)</br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>', 'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first appears in spring as reddish areas on developi

In [15]:
import random
import re

sender = random.randint(0, 100000)

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(questions):
    
    results = []
    for i, q in enumerate(tqdm.tqdm(questions)):

        DATA['message'] = q
        try:
            response = requests.post(URL, json = DATA)
            if response.status_code != 200:
                print(f'Error: Service at {URL} is unavailable, exit.')
                exit()

        except Exception as e:
            print(f'Error: Exception at posting question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        try:
            r = json.loads(response.text)
            success = False
            for link in r:
                if 'custom' in link:
                    success = True
                    r = link['custom']
                    break
        except Exception as e:
            print(f'Error: Failed on parsing response on question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        result = []
        if success:
           
            try:
                r = r['data']
                if len(r) == 0:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response on question - "{q}", exit. . {type(e).__name__}: "{e}".')

            for e in r:
                title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
                link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
                result.append((title, link))
            

            DATA['message'] = '/intent_affirm'
            try:
                response = requests.post(URL, json = DATA)
                if response.status_code != 200:
                    print(f'Error: Service at {URL} is unavailable, exit.')
                    exit()
            except Exception as e:
                print(f'Error: Exception at posting affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
            
            try:
                r = json.loads(response.text)
                r = r[0]
                r = r['text']
                if 'Anything else I can help with?' != r:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response of affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
        else:
            print(f'No results for question - "{q}"')
            result.append(('', ''))
            if (i+1)%5 == 0:
                print(f'Finished {i+1} questions...')
        
        results.append(result)
    
    print(f'Finished querying all questions for scoring')

    return results


In [16]:
import pandas as pd

FINAL_DATA = './data/transformed/result.pkl'

df = pd.read_pickle(FINAL_DATA)

df.sample(10)

,Question,ExpectedAnswer,URL,Source
31,We have tried many things to get rid of pantry...,NaN,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7452.htm...,AE
39,I have a problem with fruit flies laying their...,NaN,[https://www2.ipm.ucanr.edu/agriculture/canebe...,AE
30,I notice cobwebs in the middle of my rubrum ma...,NaN,[http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT...,AE
44,How do I deter deer from my garden?,[Pest Notes: Deer],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74117.html],UCIPM
36,How do you set a gopher trap?,"[Pest Notes: Pocket Gophers, video library]",[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7433.htm...,UCIPM
38,What do scale insects look like?,[Pest Notes: Scales],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7408.html],UCIPM
60,I have 3 peach trees that are showing discolor...,NaN,[https://www2.ipm.ucanr.edu/agriculture/peach/...,AE
48,What's the best way to get rid of weeds in a l...,[Pest Notes: Weed Management in landscapes],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7441.html],UCIPM
23,We have a few cucumber beetles that are eating...,NaN,"[http://ipm.ucanr.edu/PMG/r116300511.html\r, h...",AE
12,What does rust on roses look like?,[Pest Notes: Roses: Diseases and Disorders],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7463.html],UCIPM


In [17]:
answers     = df['URL'].values.tolist()
results     = run_tests(df['Question'])

100%|██████████| 97/97 [02:54<00:00,  1.80s/it]

Finished querying all questions for scoring


In [19]:
scores = []
for i, r in enumerate(results):
    answer = answers[i]
    topn = [False, False, False, False]
    for i1, r1 in enumerate(r):
        if r1[1].split('?')[0] in answer:
            if i1 == 0:
                topn[0] = True
            if i1 < 3:
                topn[1] = True
            if i1 < 5:
                topn[2] = True
            topn[3] = True

    scores.append(topn)
    

In [21]:
for i, score in enumerate(scores):
    if not any(score):
        print(df['Question'].iloc[i])

What are the big black bugs on my patio at night?
What does rust on roses look like?
My rose leaves are covered with dusty orange spots. Is it rust? What can I do about it?
Why is my tree dying?
How do you keep bermudagrass from spreading in flower beds?
What insecticide gets rid of whiteflies?
An animal is eating all the plants in my garden and I don't know what it is. How do I find out what's eating my garden?
I keep a colony of springtails, and mites have gotten into my colony. How do i get rid of them so i dont loose my springtails?
Our front lawn has become denuded of grass. We thought there might be grubs ruining the grass. So, I dug around and found them. I read about the process of getting rid of them with nematodes, then re-seeding, etc. do you have agents that could come and evaluate our yard and give us an opinion on next steps? 
I started noticing some sort of bug that made webs in my oak tree. It seemed to be getting into the little branches and killing the rest of the bra

In [6]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 27  (27.84%)
Top 3 : 30  (30.93%)
Top 5 : 35  (36.08%)
Top 10: 40  (41.24%)


In [6]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 27  (27.84%)
Top 3 : 30  (30.93%)
Top 5 : 34  (35.05%)
Top 10: 39  (40.21%)


In [12]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 28  (28.87%)
Top 3 : 35  (36.08%)
Top 5 : 38  (39.18%)
Top 10: 43  (44.33%)
